In [3]:
#import relevant libraries
import numpy as np
import scipy as sp
from scipy.stats import norm

# MC Pricing

In [4]:
#import relevant libraries
import numpy as np

def MC_pricing(S, T, K, r, sigma, mc_step_count, simulation_count, type):
    """
    S: Current price
    T: Time to maturity
    K: Strike Price
    r: Risk free rate
    sigma: volatility
    """
    #Each time a price is generated a we will run 10000 simulations
    #Each simulation requires iterative multiplication of the current time steps' price by the updated exponential factor
    #We try and vectorise the simulations, but run the iterative multiplication with an explicit loop
    price_paths = np.full((simulation_count, mc_step_count+1), S)


    time_step = T/mc_step_count
    e = np.random.standard_normal(size=(simulation_count, mc_step_count)) # A sim count x mc step count array

    expoential_factor = np.exp((r - (sigma**2)/2)*time_step
                               + sigma * e * np.sqrt(time_step)
                               )                                    #This produces an sim count x mc step count array of expoential factors


    # We need to set all columns except the first one to be equal to the S times the cum product of the exponential factors array
    price_paths[:, 1:] = S *np.cumprod(expoential_factor, axis=1) # multiply along the row
    final_prices = price_paths[:, -1]

    if type == 'call':
        pay_off =  np.maximum(final_prices - K, 0)

    elif type == 'put':
        pay_off =  np.maximum(K - final_prices, 0)

    option_price = np.mean(pay_off)*np.exp(-r * T)

    return option_price


# BSM Model Pricing

In [5]:
def black_scholes_price(S,K,T,r,sigma,type):
    """
    Taken from [1]
    Inputs
    #S = Current stock Price
    #K = Strike Price
    #T = Time to maturity 1 year = 1, 1 months = 1/12
    #r = risk free interest rate
    #q = dividend yield
    # sigma = volatility 
    
    Output
    # call_price = value of the option 
    """
    
    d1 = (np.log(S/K) + (r + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma* np.sqrt(T)
    
    if type == 'call':
        
        call = S * np.exp(-r*T)* sp.stats.norm.cdf(d1) - K * np.exp(-r*T)*sp.stats.norm.cdf(d2)
        return call
    
    elif type == 'put':
         K * np.exp(-r * T) * sp.stats.norm.cdf(-d2) - S * sp.stats.norm.cdf(-d1)

# Comparison of both models

In [6]:
for sigma in [0.25, 0.5, 0.75, 1]:
    S = 100 #stock price S_{0}
    K = 110 # strike
    T = 1/2 # time to maturity
    r = 0.03 # risk free risk in annual %
    # sigma = 0.25 # annual volatility in %
    mc_step_count = 100 # time steps
    simulation_count = 10000 # number of trials

    mc_price = MC_pricing(S, T, K, r, sigma, mc_step_count, simulation_count, type='call')
   
    print('MC Price')
    print(sigma, mc_price)
  


MC Price
0.25 3.861441780856085
MC Price
0.5 10.500012630647163
MC Price
0.75 17.9162308455609
MC Price
1 24.1817398038243


In [7]:
for sigma in [0.25, 0.5, 0.75, 1]:
    S = 100 #stock price S_{0}
    K = 110 # strike
    T = 1/2 # time to maturity
    r = 0.03 # risk free risk in annual %
    # sigma = 0.25 # annual volatility in %
    bsm_price = black_scholes_price(S, T, K, r, sigma,type='call')
    print('-'*60)
    print('BSM Price')
    print(sigma, bsm_price)

------------------------------------------------------------
BSM Price
0.25 3.670319165209884
------------------------------------------------------------
BSM Price
0.5 3.6852741794050052
------------------------------------------------------------
BSM Price
0.75 3.6882742114902047
------------------------------------------------------------
BSM Price
1 3.688316648426815


In [8]:

print('='*60)
for sigma in [0.25, 0.5, 0.75, 1]:
    S = 100 #stock price S_{0}
    K = 110 # strike
    T = 1/2 # time to maturity
    r = 0.03 # risk free risk in annual %
    # sigma = 0.25 # annual volatility in %
    mc_step_count = 100 # time steps
    simulation_count = 10000 # number of trials

    mc_price = MC_pricing(S, T, K, r, sigma, mc_step_count, simulation_count, type='put')
    print(sigma, mc_price)

0.25 12.85994679316026
0.5 19.494084639223125
0.75 26.636835779703052
1 33.43046324878765


In [9]:
for r in [0.03, 0.04, 0.05, 0.06]:
    S = 100 #stock price S_{0}
    K = 110 # strike
    T = 1/2 # time to maturity
    # r = 0.03 # risk free risk in annual %
    sigma = 0.25 # annual volatility in %
    mc_step_count = 100 # time steps
    simulation_count = 10000 # number of trials

    mc_price = MC_pricing(S, T, K, r, sigma, mc_step_count, simulation_count, type='call')
    print(mc_price)

print('='*60)
for r in [0.03, 0.04, 0.05, 0.06]:
    S = 100 #stock price S_{0}
    K = 110 # strike
    T = 1/2 # time to maturity
    # r = 0.03 # risk free risk in annual %
    sigma = 0.25 # annual volatility in %
    mc_step_count = 100 # time steps
    simulation_count = 10000 # number of trials

    mc_price = MC_pricing(S, T, K, r, sigma, mc_step_count, simulation_count, type='put')
    print(mc_price)

3.690032303365152
4.139084937772435
4.047828727891188
4.175924175412585
12.798180274547148
12.080752608771096
11.725760948351832
11.347419623782706


# References used
[1] : https://www.codearmo.com/blog/pricing-options-monte-carlo-simulation-python